In [58]:
import os
import math
from time import time
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl

import dvfdata

dep_selection="All"
model_name="LigthGBM"
#dep_selection="77"


In [2]:
df=dvfdata.loadDVF_Maisons(departement=dep_selection,refresh_force=False
                           ,add_commune=False,filterColsInsee="Permutation")

Read Mutations
Read Local
Read Parcelle
Read Adresse
Make the join for DVF
Filter data:
Final Calculations


In [3]:
df_prepared=dvfdata.prepare_df(df,remove_categories=False)
# Keep only random part of all records.
df_prepared=df_prepared.sample(n=100000, random_state=42)

Prepare : filter extrem values
Prepare : drop geo categories
Prepare : update categories


In [59]:
print("Refreshing : Communes Metropoles")
folder_path=os.path.join("data_INSEE_Communes")
file_path=folder_path+"/MDB-INSEE-V2_2016.xls"
communes_insee_df=pd.read_excel(io=file_path)   
communes_insee_df = communes_insee_df.rename(columns={'CODGEO':'commune'})


Refreshing : Communes Metropoles
WARNING *** file size (57125979) not 512 + multiple of sector size (512)
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


In [90]:
communes_insee_df.columns[0:10]

Index(['commune', 'Nb Pharmacies et parfumerie', 'Dynamique Entrepreneuriale',
       'Dynamique Entrepreneuriale Service et Commerce',
       'Synergie Médicale COMMUNE', 'Orientation Economique',
       'Indice Fiscal Partiel', 'Score Fiscal', 'Indice Evasion Client',
       'Score Evasion Client'],
      dtype='object')

In [91]:
# Keep only the population
commune_size_df=communes_insee_df[['commune','Population','LIBGEO','DEP']]
commune_size_df=commune_size_df.sort_values(by='Population', ascending=False)

In [92]:
commune_size_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36677 entries, 30795 to 21501
Data columns (total 4 columns):
commune       36677 non-null object
Population    36677 non-null int64
LIBGEO        36677 non-null object
DEP           36677 non-null object
dtypes: int64(1), object(3)
memory usage: 1.4+ MB


In [95]:
commune_size_df.head(10)

,commune,Population,LIBGEO,DEP
30795,75056,2125851,Paris,75
4458,13055,797491,Marseille,13
28530,69123,445274,Lyon,69
12424,31555,390301,Toulouse,31
2019,06088,343123,Nice,06
17080,44109,270343,Nantes,44
27957,67482,263941,Strasbourg,67
13639,34172,225511,Montpellier,34
12987,33063,215374,Bordeaux,33
23388,59350,212566,Lille,59


In [98]:
# Keep cities with more than 100.000 citizens
cities_100000=commune_size_df[commune_size_df['Population']>=100000]
#city_100000

In [100]:
# Keep as well the biggest city for each departments
#stats_year_month_mean=df.groupby(['DEP']).max()
biggest_dep=commune_size_df.groupby(['DEP'], sort=False)['Population'].max()
biggest_dep

DEP
75    2125851
13     797491
69     445274
31     390301
06     343123
       ...   
07      17517
70      17152
23      14114
09      13463
48      11792
Name: Population, Length: 100, dtype: int64

In [106]:
# Get the Biggest cities by Departments
idx = commune_size_df.groupby(['DEP'])['Population'].transform(max) == commune_size_df['Population']
cities_department=commune_size_df[idx]
cities_department


,commune,Population,LIBGEO,DEP
30795,75056,2125851,Paris,75
4458,13055,797491,Marseille,13
28530,69123,445274,Lyon,69
12424,31555,390301,Toulouse,31
2019,06088,343123,Nice,06
...,...,...,...,...
2104,07010,17517,Annonay,07
29216,70550,17152,Vesoul,70
8114,23096,14114,Guéret,23
3116,09225,13463,Pamiers,09


In [123]:
# Add biggest cities for each 
#df.append(df2)

cities_department.sort_values(by='DEP', ascending=False)
cities_department.sort_values(by='Population', ascending=False)

,commune,Population,LIBGEO,DEP
30795,75056,2125851,Paris,75
4458,13055,797491,Marseille,13
28530,69123,445274,Lyon,69
12424,31555,390301,Toulouse,31
2019,06088,343123,Nice,06
...,...,...,...,...
2104,07010,17517,Annonay,07
29216,70550,17152,Vesoul,70
8114,23096,14114,Guéret,23
3116,09225,13463,Pamiers,09


In [225]:
base_path="/Users/christophenoblanc/Documents/ProjetsPython/DSSP_Projet_DVF"
#folder_path=os.path.join("data_Geo")
pd.set_option('display.max_columns', 500)
file_path=base_path+"/data_Geo/EU_Geo_Circos_Regions_departements_circonscriptions_communes_gps.csv"
df_cities_csv=pd.read_csv(file_path,sep=';',decimal=".",dtype=np.str,encoding="utf_8")
df_cities_csv = df_cities_csv.rename(columns={'code_insee':'commune'})
df_geo_cities = df_cities_csv[['commune','latitude','longitude','éloignement']]
df_geo_cities['latitude']=pd.to_numeric(df_cities_csv['latitude'].replace('NaN',''), errors='raise')
df_geo_cities['longitude']=pd.to_numeric(df_cities_csv['longitude'].replace('NaN',''), errors='raise')



/Users/christophenoblanc/opt/anaconda3/envs/dvf/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/christophenoblanc/opt/anaconda3/envs/dvf/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [237]:
#df_cities_csv.iloc[12788]
missing_geo=df_geo_cities[pd.isna(df_geo_cities['latitude'])]

In [227]:
df_geo_cities = df_geo_cities.drop_duplicates(['commune'],keep='first')

In [228]:
df_geo_cities.head(10)
#f_cities_csv.head(10)

,commune,latitude,longitude,éloignement
0,01024,46.283333,5.166667,1.21
1,01029,46.400000,5.266667,1.91
2,01038,46.333333,5.283333,1.51
3,01040,46.366667,5.050000,1.71
4,01245,46.133333,5.400000,1.01
5,01053,46.200000,5.216667,1
6,01065,46.200000,5.133333,0.73
7,01069,46.133333,5.266667,1.97
8,01072,46.166667,5.316667,1.15
9,01095,46.266667,5.416667,1.14


In [229]:
df_geo_cities[df_geo_cities['commune']=='75056']

,commune,latitude,longitude,éloignement
30893,75056,48.856697,2.351462,NaN


In [230]:
# join with biggest cities
print("Make the join for DVF")
cities_department_geo = pd.merge(cities_department, df_geo_cities, how='left', on=['commune'])

Make the join for DVF


In [231]:
cities_department_geo[80:100]

,commune,Population,LIBGEO,DEP,latitude,longitude,éloignement
80,05061,36269,Gap,05,44.566667,6.083333,0.28
81,88160,35782,Epinal,88,48.183333,6.450000,0.91
82,52448,30836,Saint-Dizier,52,48.633333,4.950000,2.34
83,15014,30554,Aurillac,15,44.916667,2.450000,0.92
84,47001,30199,Agen,47,44.200000,0.633333,1.25
85,24322,30152,Périgueux,24,45.183333,0.716667,0.55
86,40192,29529,Mont-de-Marsan,40,43.883333,-0.500000,0.79
87,61001,28917,Alençon,61,48.433333,0.083333,1.55
88,39198,24955,Dole,39,47.100000,5.500000,1.76
89,12202,23734,Rodez,12,44.333333,2.566667,1


In [203]:
cities_department_geo.loc[cities_department_geo['commune']=='75056'].latitude=12